## Rubric

Instructions: DELETE this cell before you submit via a `git push` to your repo before deadline. This cell is for your reference only and is not needed in your report. 

Scoring: Out of 10 points

- Each Developing  => -2 pts
- Each Unsatisfactory/Missing => -4 pts
  - until the score is 

If students address the detailed feedback in a future checkpoint they will earn these points back


|                  | Unsatisfactory                                                                                                                                                                                                    | Developing                                                                                                                                                                                              | Proficient                                     | Excellent                                                                                                                              |
|------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------|
| Data relevance   | Did not have data relevant to their question. Or the datasets don't work together because there is no way to line them up against each other. If there are multiple datasets, most of them have this trouble | Data was only tangentially relevant to the question or a bad proxy for the question. If there are multiple datasets, some of them may be irrelevant or can't be easily combined.                       | All data sources are relevant to the question. | Multiple data sources for each aspect of the project. It's clear how the data supports the needs of the project.                         |
| Data description | Dataset or its cleaning procedures are not described. If there are multiple datasets, most have this trouble                                                                                              | Data was not fully described. If there are multiple datasets, some of them are not fully described                                                                                                      | Data was fully described                       | The details of the data descriptions and perhaps some very basic EDA also make it clear how the data supports the needs of the project. |
| Data wrangling   | Did not obtain data. They did not clean/tidy the data they obtained.  If there are multiple datasets, most have this trouble                                                                                 | Data was partially cleaned or tidied. Perhaps you struggled to verify that the data was clean because they did not present it well. If there are multiple datasets, some have this trouble | The data is cleaned and tidied.                | The data is spotless and they used tools to visualize the data cleanliness and you were convinced at first glance                      |


# COGS 108 - Data Checkpoint

## Authors

Instructions: REPLACE the contents of this cell with your team list and their contributions. Note that this will change over the course of the checkpoints

This is a modified [CRediT taxonomy of contributions](https://credit.niso.org). For each group member please list how they contributed to this project using these terms:
> Analysis, Background research, Conceptualization, Data curation, Experimental investigation, Methodology, Project administration, Software, Visualization, Writing – original draft, Writing – review & editing

Example team list and credits:
- Alice Anderson: Conceptualization, Data curation, Methodology, Writing - original draft
- Bob Barker:  Analysis, Software, Visualization
- Charlie Chang: Project administration, Software, Writing - review & editing
- Dani Delgado: Analysis, Background research, Visualization, Writing - original draft

## Research Question

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback



## Background and Prior Work

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

## Hypothesis


Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback


## Data

### Data overview

Instructions: REPLACE the contents of this cell with descriptions of your actual datasets.

For each dataset include the following information
- Dataset #1
  - Dataset Name:
  - Link to the dataset:
  - Number of observations:
  - Number of variables:
  - Description of the variables most relevant to this project
  - Descriptions of any shortcomings this dataset has with repsect to the project
- Dataset #2 (if you have more than one!)
  - same as above
- etc

Each dataset deserves either a set of bullet points as above or a few sentences if you prefer that method.

If you plan to use multiple datasets, add a few sentences about how you plan to combine these datasets.

In [ ]:
# Run this code every time when you're actively developing modules in .py files.  It's not needed if you aren't making modules
#
## this code is necessary for making sure that any modules we load are updated here 
## when their source code .py files are modified

%load_ext autoreload
%autoreload 2

In [ ]:
# Setup code -- this only needs to be run once after cloning the repo!
# this code downloads the data from its source to the `data/00-raw/` directory
# if the data hasn't updated you don't need to do this again!

# if you don't already have these packages (you should!) uncomment this line
# %pip install requests tqdm

import sys
sys.path.append('./modules') # this tells python where to look for modules to import

import get_data # this is where we get the function we need to download data

# replace the urls and filenames in this list with your actual datafiles
# yes you can use Google drive share links or whatever
# format is a list of dictionaries; 
# each dict has keys of 
#   'url' where the resource is located
#   'filename' for the local filename where it will be stored 
datafiles = [
    { 'url': 'https://raw.githubusercontent.com/fivethirtyeight/data/refs/heads/master/airline-safety/airline-safety.csv', 'filename':'airline-safety.csv'},
    { 'url': 'https://raw.githubusercontent.com/fivethirtyeight/data/refs/heads/master/bad-drivers/bad-drivers.csv', 'filename':'bad-drivers.csv'}
]

get_data.get_raw(datafiles,destination_directory='data/00-raw/')

### Dataset #1: Student Performance Dataset

Source: Kaggle - Student_performance_data 
Link: https://www.kaggle.com/datasets/rabieelkharoua/students-performance-dataset

This dataset contains individual level student records describing demographic characteristics, study habits, parental involvement, extracurricular participation, and academic performance. Each row represents one student, identified by a unique Student ID ranging from 1001 to 3392. The dataset includes approximately 2,392 student observations and 15 variables. 

Outcome Variable: 
The main outcome variable for this project is GPA (Grade Point Average) on scale from 2 to 4. Higher values indicate stronger academic performance. GPA is influenced by study habits, parental involvement and extracurricular engagement. 

Although the dataset also includes GradeClass, which categorizes GPA into letter grades (A-F), we will primarily model GPA as a continuous variable for regression. GradeClass is derived directly from GPA and therefore will not be used as a predictor. 

Demographic variables: These variables allow us to examine socioeconomic and demographic influences on academic performance.

- Age: 15–18 years
- Gender: Binary encoded (0 = Male, 1 = Female)
- Ethnicity: 0: Caucasian | 1: African American | 2: Asian | 3: Other 
- Parental Education - Highest parental education level: 
0: None | 1: High School | 2: Some College | 3: Bachelor’s | 4: Higher

Study Habits: These variables directly relate to our hypothesis that stronger academic habits predict higher GPA.

- StudyTimeWeekly: Weekly study time in hours (0–20 hours)
- Absences: Number of absences during the school year (0–30)
- Tutoring: Participation in tutoring (0 = No, 1 = Yes)

Parental involvement: This variable helps evaluate whether family engagement contributes to student outcomes.

- 0: None | 1: Low | 2: Moderate | 3: High | 4: Very High

Extracurricular: These variables allow us to test whether time spent outside academics positively or negatively relates to GPA.

- Each is binary (0 = No, 1 = Yes).
- Extracurricular, Sports, Music, Volunteering

Dataset Limitations & Concerns

1. Age Range (15–18 years): The dataset appears to represent high school students rather than higher-education students. While our research question refers to higher education, this dataset still allows us to explore relationships between habits and academic performance in a structured academic environment.

2. Self-Reported Behavior: Study time and absences may be self-reported and subject to reporting bias.

3. Categorical Encoding: Several variables are numerically coded but represent categories. These must be treated as categorical variables during modeling to avoid incorrect assumptions of linear relationships.

4. Derived Variable (GradeClass): Because GradeClass is calculated directly from GPA, including both in modeling would introduce leakage. Therefore, GPA will be used as the regression outcome.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

RAW_PATH = Path("data/00-raw")
INTERIM_PATH = Path("data/01-interim")
PROCESSED_PATH = Path("data/02-processed")

INTERIM_PATH.mkdir(parents=True, exist_ok=True)
PROCESSED_PATH.mkdir(parents=True, exist_ok=True)

#Load dataset
df_raw = pd.read_csv(RAW_PATH / "Student_performance_data_.csv")
df = df_raw.copy()

print("First 5 rows")
display(df.head())

#Is the dataset tidy?
print("\nDataset shape:")
print("Rows (observations):", df.shape[0])
print("Columns (variables):", df.shape[1])

print("\nColumn names:")
print(df.columns)

#Missing data Analysis
missing_counts = df.isna(),sum()
missing_percent = df.isna().mean() * 100

missing_summary = pd.DataFrame({
    "Missing Count" : missing_counts,
    "Missing Percent" : missing_percent
})

print("\nMissing Data Summary:")
display(missing_summary)

# Suspicious Value Checks 
print("\nSummary statistics:")
display(df.describe())

print("\nChecking for impossible GPA values:")
display(df[(df["GPA"]<0) | (df["GPA"]>4)])

print("\nChecking for negative StudyTimeWeekly:")
display(df[df["StudyTimeWeekly"]<0])

print("\nChecking for negative Absences:")
display(df[df["Absences:"]<0])

# Convert categorical vars
categorical_cols = [
    "Gender", "Ethnicity", "ParentalEducation",
    "Tutoring",  "ParentalSupport", 
    "Extracurricular", "Sports", "Music", "Volunteering",
    "GradeClass"
]

for col in categorical_cols:
    df[col] = df[col].astype("category")

df_clean = df.drop(columns=["GradeClass"])

df_raw.to_csv(INTERIM_PATH / "students_interim.csv", index=False)
df_clean.to_csv(PROCESSED_PATH / "students_processed.csv", index=False)

print("\nFinal cleaned dataset shape:")
print(df_clean.shape)

### Dataset #2 

See instructions above for Dataset #1.  Feel free to keep adding as many more datasets as you need.  Put each new dataset in its own section just like these. 

Lastly if you do have multiple datasets, add another section where you demonstrate how you will join, align, cross-reference or whatever to combine data from the different datasets

Please note that you can always keep adding more datasets in the future if these datasets you turn in for the checkpoint aren't sufficient.  The goal here is demonstrate that you can obtain and wrangle data.  You are not tied down to only use what you turn in right now.

In [ ]:
## YOUR CODE TO LOAD/CLEAN/TIDY/WRANGLE THE DATA GOES HERE


## Ethics

A. Data Collection
    
A.1 Informed Consent 
Considered ✓
We will only use publicly available datasets that were collected with appropriate consent for academic research. We will review dataset documentation to confirm ethical collection practices before any data. 

A.2 Collection bias 
Considered ✓
Survey-based student datasets may contain self-selection bias. Students who choose to participate may differ from those who do not. We acknowledge that this may limit generalizability and will clearly state this limitation in our report.

A.3 Limit PII exposure 
Considered ✓
We will only use datasets that are anonymized and do not include personally identifiable information (PII). Identity-level data such as names, emails, or IDs are not necessary for our analysis and will not be used.

A.4 Downstream bias mitigation
Considered ✓
If demographic variables are included, we will use them carefully to test for fairness rather than to reinforce stereotypes.

B. Data Storage

B.1 Data Security 
Considered ✓
We will not share raw data beyond what is publicly available from the original source.

B.2 Right to be forgotten
Considered ✓
Because we are not collecting new data and will only use anonymized public datasets, individual removal requests are not applicable to this project.

B.3 Data Retention Plan
Considered ✓
The dataset will only be used for the duration of this course project. We do not plan to retain or reuse the data beyond academic purposes.

C. Analysis
    
C.1 Missing Perspectives
Considered ✓
We recognize that our team may not represent all student experiences. We rely on prior research and will clearly state limitations in interpretation.

C.2 Dataset Bias
Considered ✓
We will examine variable distributions and discuss possible biases and confounding factors.

C.3 Honest Representation 
Considered ✓
All visualizations and statistical summaries will accurately represent the data. We will avoid misleading scales and clearly label all figures.

C.4 Privacy in Analysis 
Considered ✓
We will not display or analyze any personally identifiable information.

C.5 Auditability
Considered ✓
All data cleaning and modeling steps will be documented in our notebook to ensure transparency.

D. Modeling

D.1 Proxy Discrimination 
Considered ✓
We will examine whether any variables may unintentionally act as proxies for protected characteristics and interpret results carefully.

D.2 Fairness Across Groups 
Considered ✓
We will evaluate whether predictive performance differs across demographic groups. 

D.3 Metric Selection 
Considered ✓
We will use appropriate regression metrics and explain why they were selected. 

D.4 Explainability 
Considered ✓
We will use interpretable methods and clearly explain results in accessible language.

D.5 Communicate Limitations 
Considered ✓
We will clearly state that our findings show associations, and discuss dataset and modeling limitations.

E. Deployment

E.1 Monitoring and Evaluation 
Considered ✓
This is an academic project and will not be deployed in a real-world setting.

E.2 Redress
Considered ✓
Since the model will not be used for real decision-making, direct harm is unlikely. 

E.3 Roll back 
Considered ✓
The model exists only within our project notebook and can be easily modified or removed.

E.4 Unintended Use 
Considered ✓
We will clearly state that this model is for educational purposes only and should not be used for high-stakes decisions about students.

## Team Expectations 

Communication: 
- We will meet once per week to discuss progress and assign tasks. 
- We communicate primarily through Discord.
- Members will respond to messages within 24 hours unless they inform the group ahead of time. 
- Important decisions will be written in our group chat, shared document or GitHub repo. 

Work Distribution: 
- Tasks will be divided based on strengths and interests (data cleaning, EDA, modeling, writing, visuals).
- Every member will contribute to both coding and writing. 
- Darts must be shared at least 48 hours before deadlines so everyone can review. 
- No section will be submitted without full group agreement. 

Accountability: 
- Each member is responsible for completing assigned tasks by the agreed deadline. 
- If someone cannot complete their work, they must inform the group as soon as possible.
- If participation becomes uneven, we will address it respectfully and early. 

Conflict Resolution:
- We agree to communicate respectfully, even when we disagree
- If conflicts arise, we will first try to resolve them as a group. 
- If necessary, we will consult a TA for support. 

GitHub & Collaboration: 
- Changes will be clearly labeled with descriptive commit messages. 
- We will not overwrite each other’s work without discussion. 

Academic Integrity: 
- All code and writing will be properly cited. 
- We will not copy from other groups or online submissions. 



## Project Timeline Proposal

Instructions: Replace this with your timeline.  **PLEASE UPDATE your Timeline!** No battle plan survives contact with the enemy, so make sure we understand how your plans have changed.  Also if you have lost points on the previous checkpoint fix them